In [5]:
import requests
import json
import pandas as pd
import numpy as np

headers = {
  "Client-Id": '740209',
  "Api-Key": '73615d31-68e7-4c00-a008-e53b9d08d600'
}

In [7]:



def send_product(sku_arr: list[list[int]]):
    response = requests.post('https://api-seller.ozon.ru/v1/product/import-by-sku', json={
        "items": [
            {
                "sku": sku,
                "name": str(sku),
                "offer_id": str(sku),
                "currency_code": "RUB",
                "old_price": "2590",
                "price": "2300",
                "premium_price": "2200",
                "vat": "0.1"
            }
            for sku in sku_arr
        ]
    }, headers=headers)
    print(response)
    result = response.json()
    print(result)


df = pd.read_csv('dataset.csv')
df.head()
print(len(df))
df = df.drop_duplicates('categoryId')
print(len(df))
df = df['sku']
for sku_arr in np.array_split(df, len(df) // 500 + 1):
    send_product(sku_arr)


20749
5907
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}


c:\Python310\lib\site-packages\numpy\lib\shape_base.py:790: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))


<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'code': 7, 'message': 'Invalid Api-Key, please contact support', 'details': []}
<Response [403]>
{'c

In [8]:
# Read raw json and puts in df
def prepare_json(path: str):
    obj = json.loads(open(path, 'r', encoding='utf-8').read())
    data = []
    for key in obj:
        row = obj[key]
        if row['title'] is not None and row['description'] is not None and row['category'] is not None and row['hierarchy'] is not None:
            if row['title'].strip() != '' and row['description'].strip() != '' and row['category'].strip() != '' and row['hierarchy'].strip() != '':
                tree = row['hierarchy'].split('/')
                row['category'] = str([tree[min(5, len(tree) - 1)]])
                # row['category'] = str([row['category']])
                row['category'] = str(tree[:3])

                data.append(obj[key])

    df = pd.json_normalize(data)
    df = df.rename(columns={"title": "titles",
                   "category": "terms", 'description': 'summaries'})
    df = df[[
        'titles', 'summaries', 'terms', 'sku', 'categoryId', 'url'
    ]]
    return df

# Fixes json to final dataset


def fix_categories(dataset: str, out: str):
    result = {}
    body = {
            "filter": {
                "visibility": "ALL"
            },
            "limit": 1000,
            "sort_dir": "ASC"
        }
    while True:
        
        response = requests.post(
            'https://api-seller.ozon.ru/v3/products/info/attributes',
            json=body,
            headers=headers)
        print(response)
        if response.status_code != 200:
            break
        response = response.json()
        last = response['last_id']
        print(last)
        body['last_id'] = last
        # result.extend([
        #     {
        #         'trueCategoryId': str(old_dict['category_id']),
        #         'sku': int(old_dict['name'])
        #     }
        for old_dict in response['result']:
            if old_dict['name'].isdecimal():
                result[str(old_dict['name'])] = str(old_dict['category_id'])
        # ])

        print(len(result))
        
    df = prepare_json(dataset)
    print(len(df))
    # df = df.drop_duplicates('categoryId')
    df['trueCategoryId'] = [result.get(str(i), None) for i in df['sku']]
    # df = df.join(pd.json_normalize(result).set_index('sku'), on='sku')
    print(len(df))
    # inject categories tree
    tree = requests.post(
        'https://api-seller.ozon.ru/v2/category/tree', headers=headers)
    print(tree)
    tree = tree.json()['result']

    tree_map = {}
    stack = []
    def dfs(dict,
            stack: list[any],
            start=False):
        children = dict
        if not start:
            children = dict['children']
            
            # full
            # stack.append({
            #     'category_id': dict['category_id'],
            #     'title': dict['title']
            # })
            # titles only
            stack.append(
                dict['title']
            )
            
            
        if len(children) == 0:
            tree_map[str(dict['category_id'])] = json.dumps(stack.copy(), ensure_ascii=False)
        else:
            for item in children:
                dfs(item, stack)
        if not start: stack.pop()
    dfs(tree, stack, True)
    
    df['trueHierarchy'] = [tree_map.get(i, None) for i in df['trueCategoryId']]
        
    
    # df = df[df['trueHierarchy'].notnull()]
    json.dump(tree, open('./parse/categories_tree.json', 'w+', encoding='utf-8'),
              indent=2, ensure_ascii=False)
    df.to_csv(out)
    json.dump(tree_map,
              open('./sus.json', 'w+', encoding='utf-8'),
              indent=2, ensure_ascii=False)
    json.dump(result,
              open('./result.json', 'w+', encoding='utf-8'),
              indent=2, ensure_ascii=False)
    return df


df = fix_categories(
    './parse/products_data/found.json',
    './dataset.csv')
print(len(df))
df.head()


<Response [403]>
26060
26060
<Response [403]>


KeyError: 'result'

In [11]:
df = pd.read_csv('dataset.csv')
df.head()
print(len(df))
df = df.drop_duplicates('categoryId')
df = df.drop_duplicates('trueCategoryId')
print(len(df))
# df = df['sku']
# for sku_arr in np.array_split(df, len(df) // 500 + 1):
#     send_product(sku_arr)

20749
741
